In [15]:
import soundscapy as sspy
from rpy2 import robjects
from rpy2.robjects.conversion import localconverter
from soundscapy.spi._r_wrapper import conversion_rules
from soundscapy.spi.ks_test import ks2d2s
from soundscapy.spi.msn_params import DirectParams, CentredParams

import numpy as np

### Directly call R sn

In [16]:
with localconverter(conversion_rules):
    r_yM = robjects.r("""
        library(sn)
        Sigma <- matrix(c(1, 0.4, 0.4, 0.6), 2, 2)
        cpM <- list(mean=c(0,-1), var.cov=Sigma, gamma1=c(0, -0.6))
        dpM <- cp2dp(cpM, family="SN")
        dpM
        """)

r_yM

o{'beta': [ 0.         -0.13389297], 'Omega': [[1.         0.4       ]
 [0.4        1.35014139]], 'alpha': [  3.66054584 -10.63348162], }

In [17]:
from soundscapy.spi._r_msnparams_wrapper import RmsnParams

dp = DirectParams.from_r(RmsnParams(r_yM))
dp

DirectParams(xi=[ 0.         -0.13389297], omega=[[1.         0.4       ]
 [0.4        1.35014139]], alpha=[  3.66054584 -10.63348162])

In [37]:
from soundscapy.spi._r_wrapper import get_r_session

_, sn, _, _ = get_r_session()
dp_r = dp._pyrobj.py2rpy()
dp_r

beta,[14]
Omega,[14]
alpha,[14]


In [56]:
with localconverter(conversion_rules):
    # dp.omega = np.array([[1, 0.4], [0.4, 0.6]])
    # dp_r = dp._pyobj.py2rpy()
    # dp_r_omega = robjects.r.matrix(dp_r.omega)
    # dp_r_omega
    dp_omega_r = robjects.r.matrix(dp.omega, nrow=2, ncol=2)

dp_omega_r

array([[1.        , 0.4       ],
       [0.4       , 1.35014139]])

In [13]:
import rpy2.rlike.container as rlc
from dataclasses import dataclass


class RmsnParams(rlc.OrdDict):
    """
    R wrapper for the msnParams class in the sn package.
    """

    __rname__ = "msnParams"
    __rpackage__ = "sn"

    def __init__(self, _pyrobj: rlc.OrdDict):
        super().__init__(_pyrobj)
        self._pyrobj = _pyrobj

        if all(param in self._pyrobj for param in ["beta", "Omega", "alpha"]):
            self._param_type = "dp"
        elif all(param in self._pyrobj for param in ["mean", "sigma", "skew"]):
            self._param_type = "cp"
        else:
            raise ValueError(f"Invalid parameters. Contains: {self._pyrobj.keys()}")

    def rpy2py(self):
        """
        Convert the RmsnParams object to a Python dictionary.
        """
        return {key: self._pyrobj[key] for key in self._pyrobj.keys()}


dp = RmsnParams(r_yM)
sn.dp2cp(dp._pyrobj.py2rpy())

AttributeError: 'OrdDict' object has no attribute 'py2rpy'

In [ ]:
cp = CentredParams(
    mean=np.array([0, -1]),
    sigma=np.array([[1, 0.4], [0.4, 0.6]]),
    skew=np.array([0, -0.6]),
)
